In [1]:
# import files from Steve
import pandas as pd
import numpy as np

proficiency_path = './data/ga/proficiency/'

df = pd.read_csv(proficiency_path + 'ga_proficiency.csv')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df = df.rename(columns={'group':'group_state'})

In [6]:
# convert string fields
print(df.dtypes)
df['year'] = df['year'].astype(str)
df['proficient_tf'] = df['proficient_tf'].astype(bool)
df['grade'] = df['grade'].astype(str)
df['num_tested'] = df['num_tested'].astype('float64')
print(df.dtypes)

district_id           object
district              object
school_id             object
school                object
year                   int64
grade                  int64
group_state           object
subject               object
num_tested             int64
performance_level     object
num_at_level         float64
pct_at_level         float64
proficient_tf          int64
dtype: object
district_id           object
district              object
school_id             object
school                object
year                  object
grade                 object
group_state           object
subject               object
num_tested           float64
performance_level     object
num_at_level         float64
pct_at_level         float64
proficient_tf           bool
dtype: object


In [7]:
# drop null values rows
print(df.shape)
df = df.dropna(subset=['num_at_level', 'num_tested'])
print(df.shape)
df = df[df['num_tested'] != 0]
print(df.shape)

(6982088, 13)
(6814860, 13)
(6814860, 13)


In [10]:
# filter to only proficient rows
df = df[df.proficient_tf]

In [13]:
# roll up to single row
grouped_by = ['year', 'district_id', 'district', 'school_id', 'school', 'grade', 'group_state', 'subject', 'proficient_tf']
print(df.shape)
df1 = df.groupby(grouped_by, as_index=False)['num_at_level'].sum()
print(df1.shape)
df2 = df.groupby(grouped_by, as_index=False)['num_tested'].sum()
print(df2.shape)
df = df1.merge(df2, on=grouped_by)
print(df.shape)
del df1
del df2

(3407430, 13)
(1703715, 10)
(1703715, 10)
(1703715, 11)


In [18]:
# create All Grades roll-up
grouped_by = ['year', 'district_id', 'district', 'school_id', 'school', 'group_state', 'subject', 'proficient_tf']
print(df.shape)
df1 = df.groupby(grouped_by, as_index=False)['num_at_level'].sum()
print(df1.shape)
df2 = df.groupby(grouped_by, as_index=False)['num_tested'].sum()
print(df2.shape)
df_grades = df1.merge(df2, on=grouped_by)
print(df_grades.shape)
del df1
del df2

df_grades['grade'] = 'All'

df = df.append(df_grades, ignore_index=True, sort=True)
print(df.shape)

(1703715, 12)
(754123, 9)
(754123, 9)
(754123, 10)
(2457838, 12)


In [19]:
df['pct_at_level'] = df['num_at_level'] / df['num_tested']

In [20]:
print(df.head(5))

         district district_id grade   group_state  num_at_level  num_tested  \
0  Appling County         601    10  All Students           2.0        20.0   
1  Appling County         601    10  All Students          78.0       372.0   
2  Appling County         601    10  All Students          84.0       434.0   
3  Appling County         601    10  All Students           3.0        74.0   
4  Appling County         601    10  All Students           1.0        30.0   

   pct_at_level  proficient_tf                      school school_id  \
0      0.100000           True  Appling County High School      0103   
1      0.209677           True  Appling County High School      0103   
2      0.193548           True  Appling County High School      0103   
3      0.040541           True  Appling County High School      0103   
4      0.033333           True  Appling County High School      0103   

                                subject  year  
0  9th Grade Literature and Composition  201

In [22]:
# export file
df.to_csv('./data/finalized/ga_proficiency.csv', index=False)